In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import sklearn
import matplotlib.pyplot as plt
import optuna


In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from xgboost.sklearn import XGBRegressor

In [4]:
# data preprocessing

df = pd.read_csv("/kaggle/input/stock-index/0f411c708e55af442eafb33bfb7ee7585f5b0211a52d9ccc4287a23d8d6abe76_STOCK_INDEX.csv")
df = df.drop("Adj Close", axis =1)

# impute NA vals with mean of upper and lower
for i in range(len(df)):
    if(sum(df.iloc[i].isna())):
        a = (df.iloc[i+1, 1:6] + df.iloc[i-1, 1:6])/2
        df.iloc[i, 1:6] = a


# trainset for imputation purposes
df1 = df.iloc[732:,:] # approach 1- just drop it
for i in np.where(df1["Volume"]==0)[0]:
    df1.iloc[i, 5] = np.median(df1["Volume"].iloc[(i-5):(i+5)])
    print(df1.iloc[i, 5])
    
    
    
# impute 0 values of volume in the initial thingy
impmodel = CatBoostRegressor(verbose = False)
impmodel.fit(df1.iloc[:, 1:5] , df1.iloc[:,5])

for i in range(len(df)):
    if df.iloc[i,5] == 0:
        df.iloc[i,5] = impmodel.predict(df.iloc[i,1:5])


168400.0
191700.0
228450.0
211300.0
210950.0
233650.0
251350.0
308550.0
405950.0
325550.0
293200.0
283650.0
298050.0
299475.0
501900.0
581250.0
521100.0
506900.0


These ones are fairly intuitive
* Moving averages
* Exponential moving averages
* Daily Change %
* 

### Williams %R

Between 0 and -100 apparently

Lookback period conventionally 14 days

(max(high)- close)/(max(high- min(low))

In [13]:
def williams(input):
    mxh = max(input["High"].iloc[37:51])
    mnl = min(input["Low"].iloc[37:51])
    cl = input["Close"].iloc[50]
    
    ans = (mxh - cl)/(mxh - mnl)
    return -100*ans
    

In [14]:
williams(df.iloc[0:51])

-1.6462273289191163

### Stochastic Oscillator
100*(close - L14)/(H14- L14)

In [15]:
def stochosc(input):
    mxh = max(input["High"].iloc[37:51])
    mnl = min(input["Low"].iloc[37:51])
    cl = input["Close"].iloc[50]
    
    ans = (cl - mnl)/(mxh - mnl)
    return 100*ans
    

In [16]:
stochosc(df.iloc[0:51])

98.35377267108089

### ROC: Price Rate of Change



# Hmph

In [ ]:
def all_features(inputt):
    assert len(inputt) == 50

    df = pd.DataFrame(inputt)  # Convert input to DataFrame

    basic = pd.DataFrame([inputt.drop("Date", axis =1).to_numpy().flatten()])
    ma_10 = pd.DataFrame([ma10(inputt).values])  # Calculate moving average using all rows
    ma_20 = pd.DataFrame([ma20(inputt).values])  # Calculate moving average using all rows
#     ma_30 = pd.DataFrame([ma30(inputt).values])
#     ema_10 = pd.DataFrame([ema10(inputt).values])  # Calculate exponential moving average using all rows
#     ema_20 = pd.DataFrame([ema20(inputt).values])  # Calculate exponential moving average using all rows
    
    return pd.concat([basic, ma_10, ma_20], ignore_index=True, axis=1)